In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("datos_perfumes_primor.csv")

In [3]:
df = df.drop('Unnamed: 0', axis=1)

In [4]:
df.insert(0, 'tienda', "primor")

In [5]:
def convertir_a_minusculas(valor):
    if isinstance(valor, str):
        return valor.lower()
    return valor

df = df.map(convertir_a_minusculas)

In [6]:
df['categoria'] = df['categoria'].str.replace('perfumes para hombres', 'hombre')
df['categoria'] = df['categoria'].str.replace('perfumes de mujer', 'mujer')

In [7]:
def mapear_tipos_y_limpiar(df):    

    mapping = {
        'parfum'            : 'parfum',
        'eau de toilette'   : 'eau de toilette',
        'edt'               : 'eau de toilette',
        'eau de toillette'  : 'eau de toilette',
        'cool water'        : 'eau de toilette',
        'eau de parfum'     : 'eau de parfum',
        'edp'               : 'eau de parfum',
        'agua de perfume'   : 'eau de parfum',
        'colonia'           : 'eau de cologne',
        'edc'               : 'eau de cologne',
        'cologne'           : 'eau de cologne',
        'set'               : 'set',
        'estuche'           : 'set',
        'cofre de Navidad'  : 'set',
        'cofre'             : 'set',
        'body mist'         : 'locion corporal'
    }

    for name, tipo in mapping.items():
        df.loc[df['nombre'].str.contains(name), 'tipo'] = tipo

    for name in mapping.keys():
        df['nombre'] = df['nombre'].str.replace(name, '', regex=False)

    df['tipo'] = df['tipo'].str.strip()

    return df

mapear_tipos_y_limpiar(df)

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones
0,primor,hombre,hugo boss,boss bottled,eau de toilette,"81,70€",-56%,"35,95€","4,81",212
1,primor,hombre,davidoff,hombre,eau de toilette,"74,00€",-74%,"18,94€","4,71",118
2,primor,hombre,armani,giorgio armani perfume hombre acqua di gio homme,eau de toilette,"85,00€",-54%,"38,94€","4,93",98
3,primor,hombre,calvin klein,ck in2u for him,eau de toilette,"76,00€",-76%,"18,49€","4,70",94
4,primor,hombre,carolina herrera,212 men,eau de toilette,"74,50€",-49%,"37,95€","4,89",66
...,...,...,...,...,...,...,...,...,...,...
2318,primor,mujer,narciso rodriguez,narciso poudrée generous spray,eau de parfum,"98,00€",-35%,"84,93€",sin valoracion,sin valoraciones
2319,primor,mujer,cartier,baiser volé,set,"160,00€",sin descuento,sin precio descuento,sin valoracion,sin valoraciones
2320,primor,mujer,dolce & gabbana,light blue summer vibes,eau de toilette,sin precio,sin descuento,sin precio descuento,"5,00",2
2321,primor,mujer,guerlain,aqua allegoria nerolia vetiver de regalo,set,"103,09€",-42%,sin precio descuento,sin valoracion,sin valoraciones


In [8]:
def clean_prices(df, columns):
    
    for col in columns:
        
        df[col] = df[col].str.replace('[^0-9,]', '', regex=True) 
        df[col] = df[col].str.replace(',', '.', regex=True)
        df[col] = df[col].replace('', '0', regex=True)
        df[col] = df[col].astype(float)  

columns = ['precio_sin_desc', 'descuento', 'precio_con_desc', 'valoracion', 'num_valoraciones']

clean_prices(df, columns)

In [9]:
df['descuento'] = df['descuento'].fillna(0)

In [10]:
df['precio_con_desc'] = (df['precio_sin_desc'] * (1 - df['descuento'] / 100)).round(2)

In [11]:
columns = ['tipo', 'valoracion', 'num_valoraciones']

for col in columns:
    df[col] = df[col].replace(['sin categoria', 'sin valoracion', 'sin valoraciones'], np.nan)

In [12]:
df = df.drop_duplicates()

In [13]:
df = df[df['nombre'] != '']

In [14]:
lista_tipos = ['eau de parfum', 'eau de toilette', 'parfum', 'eau de cologne']
df = df[df["tipo"].isin(lista_tipos)].reset_index(drop=True)

In [15]:
df = df[(df["precio_sin_desc"] != 0) & (df["precio_con_desc"] != 0)]

In [16]:
df['tipo'].value_counts()

tipo
eau de toilette    586
eau de parfum      572
eau de cologne      72
parfum              43
Name: count, dtype: int64

In [17]:
df[df['tipo'] == 'sin categoria']

,tienda,categoria,marca,nombre,tipo,precio_sin_desc,descuento,precio_con_desc,valoracion,num_valoraciones


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1273 entries, 0 to 1419
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tienda            1273 non-null   object 
 1   categoria         1273 non-null   object 
 2   marca             1273 non-null   object 
 3   nombre            1273 non-null   object 
 4   tipo              1273 non-null   object 
 5   precio_sin_desc   1273 non-null   float64
 6   descuento         1273 non-null   float64
 7   precio_con_desc   1273 non-null   float64
 8   valoracion        1108 non-null   float64
 9   num_valoraciones  1108 non-null   float64
dtypes: float64(5), object(5)
memory usage: 109.4+ KB


In [19]:
df.isna().sum()

tienda                0
categoria             0
marca                 0
nombre                0
tipo                  0
precio_sin_desc       0
descuento             0
precio_con_desc       0
valoracion          165
num_valoraciones    165
dtype: int64

In [20]:
lista_tipos = ['eau de parfum', 'eau de toilette', 'parfum', 'eau de cologne']
df = df[df["tipo"].isin(lista_tipos)].reset_index(drop=True)

In [21]:
df.to_csv('datos_perfumes_primor_filtrados.csv', index=False)